In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from keras.models import Sequential
from keras import optimizers, layers

model_name = '' # TODO: model name here
model = Sequential(name=model_name)

additional_metrics = ['accuracy']
opt = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=additional_metrics)

model.summary()

In [ ]:
def show_train_history(train_hist): # pass all_test if for multipart show
    fig, axs = plt.subplots(1, 2, figsize=(20, 6))

    axs[0].plot(train_hist.history['accuracy'])
    axs[0].plot(train_hist.history['val_accuracy'])
    axs[0].set_title('model accuracy')
    axs[0].set_ylabel('accuracy')
    axs[0].set_xlabel('epoch')
    axs[0].legend(['train', 'validation'])

    # change to this for multipart training
    # for conf in all_test:
    #     axs[0].plot(all_test[conf]['val_accuracy'], label=conf)
    #     axs[0].legend()

    axs[1].plot(train_hist.history['loss'])
    axs[1].plot(train_hist.history['val_loss'])
    axs[1].set_title('model loss')
    axs[1].set_ylabel('loss')
    axs[1].set_xlabel('epoch')
    axs[1].legend(['train', 'validation'])

    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# util for OneHotEncoder
y_train, y_test = []

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_train)

categories = enc.categories_[0]
print(categories)

y_train_onehot = enc.transform(y_train).toarray()
y_test_onehot = enc.transform(y_test).toarray()

print("Shape of y_train_onehot")
print(y_train_onehot.shape)
print("Shape of y_test_onehot")
print(y_test_onehot.shape)

In [ ]:
import keras

all_test = {}
x_test, x_train = []
for bs in [0, 0.2]:
    keras.backend.clear_session() # for each train, clear session

    model = Sequential()
    model.add(keras.Input(shape=(32*32*3, )))
    n_hidden_layers = 8

    for i in range(n_hidden_layers):
        model.add(layers.Dense(288, activation='relu'))
        #tf.keras.layers.BatchNormalization()
    model.add(layers.Dropout(bs))
    model.add(layers.Dense(10, activation='softmax'))

    opt = keras.optimizers.Adam(0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    model.summary()

    train_hist = model.fit(x_train, y_train_onehot, batch_size=128,
                           epochs=25,verbose=1, validation_data=(x_test, y_test_onehot), shuffle=True)

    all_test['bs_%f' % bs] = train_hist.history  # add current test params to all_test

In [ ]:
test_result = model.evaluate(x_test, y_test_onehot)
print(f'Test result - Loss: {test_result[0]} - Accuracy: {100*test_result[1]}%')